In [18]:
import sqlite3

# Setup db & cursor
db = sqlite3.connect(":memory:")
c = db.cursor()

In [19]:
# Insert some data
c.executescript("""
CREATE table "INVENTORY" (
"ITEM" VARCHAR2(10),
"COLOR" VARCHAR2(5),
"QUANTITY" NUMERIC(3)
);
insert into INVENTORY values ('Table','Blue',124);
insert into INVENTORY values ('Table','Red',223);
insert into INVENTORY values ('Chair','Blue',101);
insert into INVENTORY values ('Chair','Red',210);
""")
db.commit()

In [20]:
# See db contents
c.execute("""
select * from INVENTORY
""")
rows = c.fetchall()
for row in rows:
    print(row)

('Table', 'Blue', 124)
('Table', 'Red', 223)
('Chair', 'Blue', 101)
('Chair', 'Red', 210)


In [25]:
# Basic aggregation
c.execute("""
select item, sum(QUANTITY), avg(QUANTITY), min(QUANTITY), max(QUANTITY)
from INVENTORY group by item
order by ITEM
""")
rows = c.fetchall()
for row in rows:
    print(row)

('Chair', 311, 155.5, 101, 210)
('Table', 347, 173.5, 124, 223)


In [26]:
# CUBE function
c.execute("""
SELECT Item, Color, SUM(Quantity)
FROM Inventory
GROUP BY CUBE (Item, Color)
""")
rows = c.fetchall()
for row in rows:
    print(row)

OperationalError: no such function: CUBE

In [27]:
# Replacement for CUBE function which doesn't exist in sqlite.
c.execute("""
SELECT Item, Color, SUM(Quantity)
FROM Inventory
GROUP BY Item, Color
UNION ALL
SELECT Item, NULL, SUM(Quantity)
FROM Inventory
GROUP BY Item
UNION ALL
SELECT NULL, Color, SUM(Quantity)
FROM Inventory
GROUP BY Color
UNION ALL
SELECT NULL, NULL, SUM(Quantity)
FROM Inventory;
""")
rows = c.fetchall()
for row in rows:
    print(row)

('Chair', 'Blue', 101)
('Chair', 'Red', 210)
('Table', 'Blue', 124)
('Table', 'Red', 223)
('Chair', None, 311)
('Table', None, 347)
(None, 'Blue', 225)
(None, 'Red', 433)
(None, None, 658)
